In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import time
import csv

pref_code = [46] #都道府県コード
block_code = [1008] #地域コード
place_name = ["小田原"]

In [ ]:
# DBファイルを保存するためのファイルパス

import sqlite3

# Github
path = '/content/'

# DBファイル名
db_name = 'github.sqlite'

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)
cur = con.cursor()

# テーブルが存在しない場合は作成
cur.execute('''CREATE TABLE IF NOT EXISTS weather
                ( TEXT, language TEXT, star INTEGER)''')

# DBへの接続を閉じる
con.commit()
con.close()

In [ ]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=%s&block_no%s&year=%s&month=%s&day=17&view=p1"

# 取ったデータをfloat型に変える(データが取れなかった場合、気象庁は"/"を埋め込んでいるため0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0

In [ ]:
if __name__ == "__main__":
    for place in place_name:
    # 最終的にデータを集めるリスト
        list = [['年月日', '降水量', '気温_平均', '気温_最高', '気温_最低', '湿度_平均', '湿度_最小', '日照時間']]
        print(place)
        index = place_name.index(place)

    # 対象期間の年月を指定
    year = 2023
    print(year)
    month = 12
    #2つの都市コードと年と月を当てはめる。
    r = requests.get(url%(pref_code[index], block_code[index], year, month))
    r.encoding = r.apparent_encoding

    # サイトごとスクレイピング
    soup = BeautifulSoup(r.text)
    # findAllで条件に一致するものをすべて抜き出す。
    # 今回の条件はtrタグでclassがmtxになっているもの。
    rows = soup.findAll('tr',class_='mtx')

    # 表の最初の1~4行目はカラム情報なのでスライスする。
    rows = rows[4:]

    # 対象期間の1行を取得
    for row in rows:
        # trのなかのtdをすべて抜き出す
        data = row.findAll('td')

        #1行の中のデータを全部取り出す。
        rowData = [] #初期化
        rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
        rowData.append(str2float(data[3].string))
        rowData.append(str2float(data[6].string))
        rowData.append(str2float(data[7].string))
        rowData.append(str2float(data[8].string))
        rowData.append(str2float(data[9].string))
        rowData.append(str2float(data[10].string))
        rowData.append(str2float(data[16].string)) 

        #次の行にデータを追加
        list.append(rowData)

    #ファイルを生成(csvファイル形式。名前は都市名)
    with open(place + '.csv', 'w',encoding="utf_8_sig") as file: #文字化け防止
      writer = csv.writer(file, lineterminator='\n')
      writer.writerows(list)
